# Limpeza e Preparação dos microdados da PNAD contínua

Este código Python é uma ferramenta para **consolidar e extrair dados específicos dos microdados da Pesquisa Nacional por Amostra de Domicílios Contínua (PNADC)**, disponibilizados pelo IBGE. Ele foi projetado para processar os arquivos de texto (`.txt`) de todos os trimestres de 2023 e gerar um único arquivo CSV (`.csv`) com as informações selecionadas, focando em características de **desemprego e educação**.

---

## Como o código funciona?

O script pode ser dividido em três partes principais: configuração, preparação de dados e execução.

### 1. Configuração

Nesta seção, definimos os parâmetros essenciais para o funcionamento do script:

* **`arquivos_txt_ibge`**: Uma lista com os nomes dos arquivos `.txt` dos microdados da PNADC para cada trimestre de 2023.
* **`arquivo_csv_saida`**: O nome que você deseja dar ao arquivo CSV final, que conterá todos os dados consolidados.
* **`colunas_selecionadas`**: Uma lista de tuplas que especifica as colunas que serão extraídas de cada arquivo `.txt`. Para cada coluna, é fornecido:
    * O nome original da coluna (conforme o dicionário de variáveis da PNADC).
    * A posição inicial da coluna no arquivo de texto (base 1).
    * O tamanho (número de caracteres) da coluna.

    Essas colunas são agrupadas por suas categorias: Identificação e Localização, Características Gerais dos Moradores, Características de Educação e Características de Trabalho (com foco em ocupação e desemprego).
* **`mapeamento_nomes_colunas`**: Um dicionário que relaciona os nomes originais das colunas (geralmente códigos como "V1023") a nomes mais descritivos e fáceis de entender, como "TIPO\_AREA". Isso torna o arquivo CSV final muito mais legível.

### 2. Preparação dos Dados

Antes de processar os arquivos, o código prepara as informações de coluna para que o pandas (uma biblioteca de manipulação de dados em Python) possa lê-las corretamente:

* Ele cria `nomes_colunas_originais` com apenas os nomes das colunas que serão usadas.
* Ele calcula as `posicoes_colunas` no formato que o pandas espera para leitura de arquivos de largura fixa (`colspecs`), convertendo a posição base 1 (humana) para base 0 (programação).

### 3. Execução do Script

Esta é a parte principal do código, onde a magia acontece:

* **Loop de Processamento**: O script itera sobre cada arquivo TXT listado em `arquivos_txt_ibge`.
* **Verificação de Existência**: Antes de tentar ler um arquivo, ele verifica se o arquivo realmente existe no diretório. Se não existir, emite um aviso e pula para o próximo arquivo.
* **Leitura do Arquivo**: Utiliza a função `pd.read_fwf()` do pandas para ler os arquivos de texto de largura fixa. É importante notar o uso de `encoding='latin-1'`, que é um tipo de codificação comum para dados do IBGE e evita erros de leitura de caracteres especiais.
* **Renomeação de Colunas**: Após a leitura, as colunas são renomeadas para os nomes mais descritivos definidos no `mapeamento_nomes_colunas`, melhorando a clareza dos dados.
* **Consolidação em CSV**:
    * Para o **primeiro arquivo** processado, o script **cria** um novo arquivo CSV (`arquivo_csv_saida`) e inclui o cabeçalho das colunas.
    * Para os **arquivos subsequentes**, ele **adiciona** os dados ao mesmo arquivo CSV, mas **sem o cabeçalho**, garantindo que o arquivo final tenha apenas um conjunto de cabeçalhos.
* **Tratamento de Erro de Memória**: O código inclui um bloco `try-except` para lidar com `MemoryError`. Se um arquivo for muito grande para ser carregado de uma vez na memória, o script tenta lê-lo em **blocos (chunks)** menores. Isso é crucial para lidar com grandes volumes de dados sem esgotar a memória RAM do computador.
* **Mensagens de Progresso e Erro**: Durante todo o processo, o script exibe mensagens informativas sobre o progresso e, caso ocorra algum erro, uma mensagem detalhada é exibida para ajudar na depuração.

---

## Para que serve este código?

Este script é extremamente útil para pesquisadores, analistas de dados e estudantes que precisam trabalhar com os microdados da PNADC do IBGE. Ele automatiza o processo de:

* **Extração seletiva**: Não precisamos carregar o arquivo `.txt` inteiro e depois selecionar as colunas, o que economiza tempo e recursos computacionais.
* **Consolidação de dados**: Facilita a análise de tendências anuais ao combinar dados de diferentes trimestres em um único arquivo.
* **Limpeza e organização**: Os nomes de colunas descritivos melhoram a compreensibilidade e a usabilidade dos dados.
* **Tratamento de grandes volumes de dados**: A capacidade de processar arquivos em chunks (pedações) permite lidar com datasets que seriam impossíveis de carregar completamente na memória.

Ao final da execução, temos um arquivo CSV consolidado (`pnadc_2023_consolidado_desemprego_educacao.csv` neste exemplo) pronto para ser importado em ferramentas de análise de dados como Excel, Tableau, Power BI ou diretamente em outros scripts Python para análises mais aprofundadas. No nosso caso, usamos o Postgree SQL para fazer as consultas de interesse para o projeto.

Esperamos que esta explicação detalhada ajude a entender o propósito e o funcionamento do código! 

In [2]:
import pandas as pd
import os # Importar para verificar a existência de arquivos

# --- Configure aqui ---

# Lista dos arquivos de microdados TXT dos 4 trimestres de 2023
# Adapte os nomes dos arquivos se forem diferentes
arquivos_txt_ibge = [
    'PNADC_012023.txt', # 1º Trimestre
    'PNADC_022023.txt', # 2º Trimestre
    'PNADC_032023.txt', # 3º Trimestre
    'PNADC_042023.txt', # 4º Trimestre
]

# Diga qual o nome do arquivo CSV que você quer criar para a saída consolidada
arquivo_csv_saida = 'pnadc_2023_consolidado_desemprego_educacao.csv'

# Defina as colunas para extrair com base no dicionário fornecido e na análise proposta.
# Formato: ('Nome da Coluna', Posição Inicial no arquivo (base 1), Tamanho)
colunas_selecionadas = [
    # Parte 1 - Identificação e Localização
    ('Ano', 1, 4), # Ano de referência
    ('Trimestre', 5, 1), # Trimestre de referência
    ('UF', 6, 2), # Unidade da Federação
    ('Capital', 8, 2), # Município da Capital
    ('RM_RIDE', 10, 2), # Região Metropolitana e Região Administrativa Integrada de Desenvolvimento
    ('V1023', 34, 1), # Tipo de área (Urbana/Rural)
    ('V1027', 35, 15), # Peso do domicílio e das pessoas (com correção de não entrevista sem calibração)
    ('V1028', 50, 15), # Peso do domicílio e das pessoas (com correção de não entrevista com calibração)

    # Parte 2 - Características Gerais dos Moradores
    ('V2005', 93, 2), # Condição no domicílio
    ('V2007', 95, 1), # Sexo
    ('V2009', 104, 3), # Idade do morador na data de referência
    ('V2010', 107, 1), # Cor ou raça

    # Parte 3 - Características de Educação
    ('V3001', 108, 1), # Sabe ler e escrever?
    ('V3002', 109, 1), # Frequenta escola?
    ('V3009A', 125, 2), # Qual foi o curso mais elevado que ... frequentou anteriormente?
    ('V3014', 135, 1), # Concluiu este curso que frequentou anteriormente?

    # Parte 4 - Características de trabalho das pessoas (foco em ocupação e desemprego)
    ('V4001', 136, 1), # Trabalhou ou estagiou remunerado em dinheiro?
    ('V4002', 137, 1), # Trabalhou ou estagiou remunerado em produtos/mercadorias?
    ('V4003', 138, 1), # Fez bico/trabalho ocasional remunerado?
    ('V4004', 139, 1), # Ajudou trabalho não remunerado de morador/parente?
    ('V4005', 140, 1), # Tinha trabalho remunerado do qual estava temporariamente afastado?
    ('V403311', 199, 1), # Número da faixa de renda para a retirada em dinheiro
    ('V403312', 200, 8), # Renda bruta mensal
    ('V4071', 376, 1), # Tomou alguma providência para conseguir trabalho (30 dias)?
    ('V4072A', 379, 1), # Qual foi a principal providência que tomou para conseguir trabalho?
    ('V4073', 380, 1), # Gostaria de ter trabalho, mesmo sem procurar?
    ('V4074A', 382, 2), # Qual foi o principal motivo de não ter tomado providência para conseguir trabalho?
    ('V4078A', 396, 1), # Qual foi o principal motivo para ... não querer(poder começar a) trabalhar na semana de referência?
]

# Dicionário de mapeamento de nomes originais para nomes mais descritivos
mapeamento_nomes_colunas = {
    'Ano': 'ANO',
    'Trimestre': 'TRIMESTRE',
    'UF': 'UF',
    'Capital': 'CAPITAL',
    'RM_RIDE': 'REGIAO_METROP_RIDE',
    'V1023': 'TIPO_AREA',
    'V1027': 'PESO_DOMIC_S_CALIB',
    'V1028': 'PESO_DOMIC_C_CALIB',
    'V2005': 'CONDICAO_DOMICILIO',
    'V2007': 'SEXO',
    'V2009': 'IDADE',
    'V2010': 'COR_RACA',
    'V3001': 'SABE_LER_ESCREVER',
    'V3002': 'FREQUENTA_ESCOLA',
    'V3009A': 'CURSO_ELEVADO_ANTERIOR', # Alterado para o nome da nova coluna
    'V3014': 'CONCLUIU_CURSO_ANTERIOR',
    'V4001': 'TRAB_REMUN_DINHEIRO',
    'V4002': 'TRAB_REMUN_PROD_MERC',
    'V4003': 'FEZ_BICO_OCASIONAL',
    'V4004': 'AJUDA_TRAB_NAO_REMUN',
    'V4005': 'AFASTADO_TRAB_REMUN',
    'V403311': 'FAIXA_RENDA',
    'V403312': 'RENDA_BRUTA_MENSAL',
    'V4071': 'PROCUROU_TRABALHO',
    'V4072A': 'PRINC_PROVID_PROCURA', # Alterado para o nome da nova coluna
    'V4073': 'GOSTARIA_TRAB_NAO_PROCURA',
    'V4074A': 'MOTIVO_NAO_PROCURA', # Alterado para o nome da nova coluna
    'V4078A': 'MOTIVO_NAO_COME_TRAB' # Alterado para o nome da nova coluna
}


# Prepara as listas para o pandas
# Usamos os nomes originais para leitura e renomeamos depois
nomes_colunas_originais = [col[0] for col in colunas_selecionadas]
posicoes_colunas = []
for col_name, start_pos_1_base, length in colunas_selecionadas:
    start_pos_0_base = start_pos_1_base - 1
    end_pos_0_base = start_pos_0_base + length
    posicoes_colunas.append((start_pos_0_base, end_pos_0_base))

# --- Execução do Script ---
print("Iniciando processamento dos arquivos da PNAD Contínua...")

# Usaremos o modo de escrita 'w' para o primeiro arquivo (cria ou sobrescreve)
# e o modo de apêndice 'a' para os arquivos seguintes.
first_file = True

for i, arquivo_txt_ibge_atual in enumerate(arquivos_txt_ibge):
    print(f"\nProcessando o arquivo: {arquivo_txt_ibge_atual} ({i+1}/{len(arquivos_txt_ibge)})...")

    if not os.path.exists(arquivo_txt_ibge_atual):
        print(f"Aviso: O arquivo '{arquivo_txt_ibge_atual}' não foi encontrado e será pulado.")
        continue # Pula para o próximo arquivo se este não existir

    try:
        # Leitura direta do arquivo (para arquivos que cabem na RAM)
        df_temp = pd.read_fwf(
            arquivo_txt_ibge_atual,
            colspecs=posicoes_colunas,
            names=nomes_colunas_originais, # Usar os nomes originais para a leitura
            encoding='latin-1' # Usar encoding correto, 'latin-1' é comum para dados do IBGE
        )
        print(f"Arquivo '{arquivo_txt_ibge_atual}' lido com sucesso! Total de {len(df_temp)} linhas.")

        # Renomear as colunas do DataFrame usando o mapeamento
        df_temp.rename(columns=mapeamento_nomes_colunas, inplace=True)
        
        if first_file:
            # Se for o primeiro arquivo, cria o CSV e escreve o cabeçalho
            df_temp.to_csv(arquivo_csv_saida, mode='w', index=False, header=True)
            first_file = False
        else:
            # Para os arquivos seguintes, anexa ao CSV sem o cabeçalho
            df_temp.to_csv(arquivo_csv_saida, mode='a', index=False, header=False)
        
        print(f"Dados de '{arquivo_txt_ibge_atual}' adicionados a '{arquivo_csv_saida}'.")

    except MemoryError:
        print(f"Erro de memória ao carregar o arquivo '{arquivo_txt_ibge_atual}' completo. Tentando carregar em chunks...")
        # --- Carregamento em Chunks para Memória Limitada ---
        chunk_size = 50000 # Experimente com 50.000, 100.000, etc.

        first_chunk_for_file = True
        for j, chunk_df in enumerate(pd.read_fwf(
            arquivo_txt_ibge_atual,
            colspecs=posicoes_colunas,
            names=nomes_colunas_originais, # Usar os nomes originais para a leitura
            chunksize=chunk_size,
            encoding='latin-1'
        )):
            print(f"Processando chunk {j+1} de '{arquivo_txt_ibge_atual}'...")
            
            # Renomear as colunas do chunk usando o mapeamento
            chunk_df.rename(columns=mapeamento_nomes_colunas, inplace=True)

            if first_file and first_chunk_for_file:
                chunk_df.to_csv(arquivo_csv_saida, mode='w', index=False, header=True)
                first_file = False
                first_chunk_for_file = False
            else:
                chunk_df.to_csv(arquivo_csv_saida, mode='a', index=False, header=False)
        print(f"Arquivo '{arquivo_txt_ibge_atual}' processado via chunks e dados adicionados a '{arquivo_csv_saida}'.")

    except Exception as e:
        print(f"Ocorreu um erro ao processar '{arquivo_txt_ibge_atual}': {e}")

print(f"\nTodos os arquivos foram processados. O arquivo consolidado '{arquivo_csv_saida}' foi criado com sucesso!")

Iniciando processamento dos arquivos da PNAD Contínua...

Processando o arquivo: PNADC_012023.txt (1/4)...
Arquivo 'PNADC_012023.txt' lido com sucesso! Total de 473335 linhas.
Dados de 'PNADC_012023.txt' adicionados a 'pnadc_2023_consolidado_desemprego_educacao.csv'.

Processando o arquivo: PNADC_022023.txt (2/4)...
Arquivo 'PNADC_022023.txt' lido com sucesso! Total de 474575 linhas.
Dados de 'PNADC_022023.txt' adicionados a 'pnadc_2023_consolidado_desemprego_educacao.csv'.

Processando o arquivo: PNADC_032023.txt (3/4)...
Arquivo 'PNADC_032023.txt' lido com sucesso! Total de 479873 linhas.
Dados de 'PNADC_032023.txt' adicionados a 'pnadc_2023_consolidado_desemprego_educacao.csv'.

Processando o arquivo: PNADC_042023.txt (4/4)...
Arquivo 'PNADC_042023.txt' lido com sucesso! Total de 473206 linhas.
Dados de 'PNADC_042023.txt' adicionados a 'pnadc_2023_consolidado_desemprego_educacao.csv'.

Todos os arquivos foram processados. O arquivo consolidado 'pnadc_2023_consolidado_desemprego_educ